In [11]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
import numpy as np
import pandas as pd
import tqdm
from datetime import date


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
#      https://www.jobs.bg/front_job_search.php?frompage=15&add_sh=1&from_hp=1&term=#paging

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    total_job_number = soup.find('td',
                            attrs={'style':
                                  'height:25px;width:220px;font-weight:500;padding-bottom:5px;'}).text.split()[-1]
    total_job_number= int(total_job_number)
    total_pages = int(total_job_number/15)+1 #round up
    return total_pages

In [63]:
# total_pages = get_pages(soup)
total_pages = 20
results =[]

for page in tqdm.tqdm(range(total_pages)):

    url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    jobs_list = soup.find_all('table',
                               attrs={'style':
                                       'width:980px;'})[0].find_all('tr')

    for job in jobs_list:
        job_date = job.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        if job_date == 'днес':
            job_date=date.today().strftime("%d.%m.%y")
        rating = job.find('span',attrs={'class':'iconed'}).text.strip()
        position = job.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = job.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
        results.append([job_date, rating, position, benefits])

  0%|          | 0/20 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

# jobs_list = soup.find_all('table',
#                            attrs={'style':
#                                    'width:980px;'})[0].find_all('tr')

In [ ]:
soup.find_all('table',
                           attrs={'style':
                                   'width:980px;'})[0].find_all('tr')

In [26]:
pd.DataFrame(results)

,0,1,2,3
0,29.04.21,,(IT) Recruitment wizard and warrior,София; Възможност за работа от вкъщи; Дистанци...
1,29.04.21,,Technical Support Engineer,София
2,29.04.21,,Java developer,София; Възможност за работа от вкъщи; Дистанци...
3,29.04.21,,Senior Customer Technical Support and Sales Ag...,София; Заплата от 1800 до 2200 BGN (Нето)
4,29.04.21,,Асистент в кухня,София
...,...,...,...,...
445,27.04.21,,Scrum Master with German,София; Възможност за работа от вкъщи
446,27.04.21,,Доставчик с личен автомобил,София; Заплата от 1600 до 2600 BGN (Нето)
447,27.04.21,,Junior Recruitment Specialist,София; Дистанционно интервю
448,27.04.21,,Junior Agent with German and English,София


In [57]:
url = f'https://www.jobs.bg/front_job_search.php?frompage=15010&add_sh=1&from_hp=1&term=#paging'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

jobs_list = soup.find_all('table',
                      attrs={'style':'width:980px;'})[1].find_all('tr')

rating = jobs_list[1].find('span',attrs={'class':'iconed'}).text.strip()

In [58]:
rating

'\uf4b1'

In [59]:
soup.find_all('table',attrs={'style':'width:980px;'})[1].find_all('tr')[0].find('span',attrs={'class':'iconed'})

<span class="iconed" style="color: #ffaa2d;"></span>